File: prices.ipynb\
Author: alexkobz\
Date: \
Task: DSRFU-298

#### Шаг 0. Импорт и получение токена

In [1]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date as d, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil

start = dt.now()
# последний день месяца
REPORT_DATE = pd.to_datetime(d.today() - timedelta(days=28) + pd.offsets.MonthEnd(n=1))
REPORT_DATE_STR = REPORT_DATE.strftime("%Y-%m-%d")
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipyk

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipyk

AttributeError: _ARRAY_API not found

In [2]:
from rudata.RuDataDF import RuDataDF
from rudata.RuDataRequest import RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


In [3]:
month_year = REPORT_DATE.strftime("%m.%Y")
isin_file = f"ISIN_{month_year}.xlsx"
Prices = pd.read_excel(f"./data/Input/{isin_file}", names=["ISIN"]).dropna().set_index("ISIN")
Prices["Date"] = REPORT_DATE
Prices[["Name", "Type", "Principal", "Currency", "Clean price pct", "Clean price", "Interest", "Price", 
        "Source", "ISIN in DB", "FX Rate", "RUB Price", "CBR discount 6D", "Discount NCC", "NCC collateral"]] = np.nan

In [7]:
import certifi
print(certifi.where())

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/certifi/cacert.pem


In [5]:
!python3 -m pip install --upgrade certifi

  Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [6]:
!python3 -V

Python 3.8.20


#### Шаг 1. RuData

##### Шаг 1.0. RuData stage

In [3]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
post start
post start
post start
post start
post start
Exception raised in FintoolReferenceData.
Cannot connect to host dh2.efir-net.ru:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')]
Retry
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/aiohttp/connector.py", line 1080, in _wrap_create_connection
    return await self._loop.create_connection(*args, **kwargs, sock=sock)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 1113, in create_connection
    transport, protocol = await self._create_connection_transport(
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.


KeyboardInterrupt



In [ ]:
AccruedInterestOnDate = RuDataDF("AccruedInterestOnDate").df
AccruedInterestOnDate.head()

In [ ]:
RUPriceHistory = RuDataDF("RUPriceHistory").df
RUPriceHistory.shape

In [ ]:
ExchangeTree = RuDataDF("ExchangeTree").df
ExchangeTree.head()

In [ ]:
MoexSecurities = RuDataDF("MoexSecurities").df
MoexSecurities.head()

In [ ]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI

isins = Prices[Prices["Price"].isna()].index.unique().tolist()
key = "EndOfDay"
url: str = getattr(DocsAPI, key).url
payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
payloads = []
for isin in isins:
    payload["isin"] = isin
    payloads.append(payload.copy())
    
async def end_of_day_rudata():
    end_of_day = []
    async with aiohttp.ClientSession(
            connector=aiohttp.TCPConnector(limit=DocsAPI.LIMIT,
                                           family=socket.AF_INET),
            trust_env=True,
            timeout=aiohttp.ClientTimeout(3600)
    ) as session:
        for i, chunk in enumerate(divide_chunks(payloads, DocsAPI.LIMIT)):
            tasks: List[asyncio.Task] = [asyncio.create_task(
                RuDataRequest(url, session).post(payload=payload)
            ) for payload in chunk]
            resAll: List[List[dict]] = [await task for task in asyncio.as_completed(tasks, timeout=600)]
            result: List[dict] = [row for task_res in resAll for row in task_res]
            end_of_day.extend(result)
    return end_of_day
    
end_of_day = asyncio.run(end_of_day_rudata())
EndOfDay = pd.DataFrame(end_of_day)
EndOfDay.head()

In [ ]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI
from time import sleep

currencies = pd.read_csv("./data/Input/currencies.csv", header=0, encoding='cp1251', sep=';', usecols=["Currency"])["Currency"].tolist()
key = "CurrencyRate"
url: str = getattr(DocsAPI, key).url
payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
currencies_rudata = []
for currency in currencies:
    payload["from"] = currency
    ans = requests.post(url, json={'from': currency, 'to': 'RUB', 'date': REPORT_DATE_STR}, headers=RuDataRequest.headers)
    result: dict = ans.json()
    result["currency"] = currency
    currencies_rudata.append(result)
    sleep(1)
CurrencyRate = pd.DataFrame(currencies_rudata)
CurrencyRate.head()

##### Шаг 1.1. Discounts

###### Шаг 1.1.1. CBR

In [ ]:
from sources.CBR_Soap import CBR_Soap

headers = {key: item for key, item in RuDataRequest.headers.items() if key != "Authorization"}
cbr_disc = CBR_Soap()\
.get_discounts(date=REPORT_DATE_STR)[['ISIN', 'от 1 месяца до 180 дней_Beg']]\
.rename(columns={'от 1 месяца до 180 дней_Beg': 'CBR discount 6D'})\
.set_index('ISIN')

In [ ]:
Prices.update(cbr_disc)
Prices['CBR discount 6D'] = Prices['CBR discount 6D'].fillna(1)

###### Шаг 1.1.2. nationalclearingcentre Рыночные риски

In [ ]:
currMarketRatesResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/fondMarketRates?action=xls&xls=1&lang=ru&date={REPORT_DATE.strftime("%d.%m.%Y")}', headers=headers)
currMarketRates = pd.read_excel(currMarketRatesResponse.content,header=2)
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)].reset_index(drop=True)
currMarketRates = currMarketRates\
.merge(MoexSecurities[['secid', 'isin']], how='left', left_on='Инструмент', right_on='secid')\
.rename(columns={'isin': 'ISIN'})\
.groupby('ISIN')\
.last()
currMarketRates = currMarketRates[currMarketRates.index.str.match('([A-Z]{2})([A-Z0-9]{9})([0-9]{1})$')]
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)]
currMarketRates=currMarketRates[['Ставка риска падения цены,%']]\
.rename(columns={'Ставка риска падения цены,%': 'Risk rate'})
currMarketRates['Discount NCC'] = currMarketRates['Risk rate'].apply(lambda x: min(30, ceil(x/np.sqrt(2)))) #Формула из п.12.1 Методики НКЦ определения риск-параметров
currMarketRates /= 100

In [ ]:
Prices.update(currMarketRates)
Prices['Discount NCC'] = Prices['Discount NCC'].fillna(1)

###### Шаг 1.1.3. nationalclearingcentre Параметры ценных бумаг

In [ ]:
securInfoResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/securInfo?action=xls&lang=ru&settleDate={REPORT_DATE.strftime("%d.%m.%Y")}', headers=headers)
securInfo = pd.read_excel(securInfoResponse.content, header=2)[['ISIN ЦБ', 'Принимается в обеспеч.']]\
.rename(columns={
    'ISIN ЦБ': 'ISIN',
    'Принимается в обеспеч.': 'NCC collateral'
})\
.dropna()\
.set_index('ISIN')
securInfo = securInfo[securInfo['NCC collateral']=='Да']

In [ ]:
Prices.update(securInfo)
Prices['NCC collateral'] = Prices['NCC collateral'].fillna('Нет')

##### Шаг 1.2. RuData transform

In [ ]:
TransformFintoolReferenceData = FintoolReferenceData[['isincode', 'nickname', 'fintooltype', 'fintoolid', 'faceftname']]\
.rename(columns={
    "isincode": "ISIN",
    "nickname": "Name",
    "fintoolid": "Fintool",
    "fintooltype": "Type", 
    "faceftname": "Currency"
})
TransformFintoolReferenceData["ISIN in DB"] = 1
TransformFintoolReferenceData["Source"] = 'RU Data (ЦЦ НРД)'
TransformFintoolReferenceData = TransformFintoolReferenceData.merge(
    AccruedInterestOnDate[['fintoolId', 'accruedInterest', 'currentFaceValue']], how="left", left_on="Fintool", right_on="fintoolId")
TransformFintoolReferenceData = TransformFintoolReferenceData\
.rename(columns={
    'accruedInterest': 'Interest', 
    'currentFaceValue': 'Principal'})\
.drop(columns=["fintoolId"])

In [ ]:
RUPriceHistoryLast = RUPriceHistory.groupby('isincode').last().rename(columns={
    'vp_pct': 'Clean price pct', 
    'vp_pc': 'Clean price',
    'acc_int': 'InterestHistory'
}).reset_index()

In [ ]:
TransformFintoolReferenceData = TransformFintoolReferenceData.merge(
    RUPriceHistoryLast[['isincode', 'Clean price pct', 'Clean price', 'InterestHistory']], how='left', left_on="ISIN", right_on="isincode"
).drop(columns=["isincode"])
TransformFintoolReferenceData["Interest"] = TransformFintoolReferenceData["Interest"].combine_first(TransformFintoolReferenceData["InterestHistory"])
TransformFintoolReferenceData["Interest"].fillna(0, inplace=True)
TransformFintoolReferenceData.drop(columns="InterestHistory", inplace=True)

In [ ]:
bonds_index = TransformFintoolReferenceData[TransformFintoolReferenceData["Type"]=='Облигация'].index

In [ ]:
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Price"] = \
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Clean price"] + \
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Interest"]

TransformFintoolReferenceData.loc[~TransformFintoolReferenceData.index.isin(bonds_index), "Price"] = \
TransformFintoolReferenceData.loc[~TransformFintoolReferenceData.index.isin(bonds_index), "Clean price"]

In [ ]:
Prices.update(TransformFintoolReferenceData.groupby("ISIN").first())

In [ ]:
exch_prices = EndOfDay.merge(
    ExchangeTree[['id', 'fullname_rus']], how='left', left_on='id_trade_site', right_on='id')[['fintoolId', 'last', 'fullname_rus']]
exch_prices["Source"] = exch_prices['fullname_rus'].apply(lambda x: 'RU Data (' + str(x) + ')')
exch_prices = exch_prices\
.rename(columns={
    'fintoolId': 'Fintool',
    'last': 'PriceEX'
})\
.drop(columns="fullname_rus")
exch_prices = exch_prices.merge(TransformFintoolReferenceData[["Fintool", "ISIN", "Type", "Principal", "Interest"]], how='inner')

In [ ]:
exch_bonds = exch_prices[exch_prices['Type']=='Облигация']
exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
exch_bonds["Price"] = exch_bonds["Clean price"] + exch_bonds["Interest"]

In [ ]:
exch_not_bonds = exch_prices[exch_prices['Type']!='Облигация']
exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
exch_not_bonds['Price'] = exch_not_bonds['PriceEX']

In [ ]:
Prices.update(exch_bonds.set_index("ISIN"))
Prices.update(exch_not_bonds.set_index("ISIN"))

In [ ]:
ResultPrices = Prices[~Prices['Price'].isna()]
Prices = Prices[Prices['Price'].isna()]

#### Шаг 2. Cbonds

##### Шаг 2.1. Bonds

In [ ]:
tradings_bonds = pd.read_excel(f'./data/Input/tradings_bonds.xls')
tradings_bonds_1 = pd.read_excel(f'./data/Input/tradings_bonds_1.xls')

tradings_bonds = pd.concat([tradings_bonds, tradings_bonds_1], axis=0)
tradings_bonds = tradings_bonds[['ISIN', 'Краткое название эмитента (rus)', 'Валюта', 'Индикативная цена', 'Номинал по еврооблигациям']]\
.rename(columns={
    'Краткое название эмитента (rus)': 'Name', 
    'Валюта': 'Currency', 
    'Индикативная цена': 'Clean price pct', 
    'Номинал по еврооблигациям': 'Principal EB'
})

In [ ]:
tradings_bonds = tradings_bonds.merge(Prices[["Interest", "Principal"]], how='inner', left_on="ISIN", right_index=True)
principal_idx = tradings_bonds[tradings_bonds['Principal']>tradings_bonds['Principal EB']].index
tradings_bonds.loc[principal_idx, 'Principal'] /= tradings_bonds.loc[principal_idx, 'Principal EB']
tradings_bonds.loc[principal_idx, 'Interest'] /= tradings_bonds.loc[principal_idx, 'Principal']

In [ ]:
Prices.update(tradings_bonds.set_index("ISIN"))

In [ ]:
tradings_nkd = pd.read_excel(f'./data/Input/tradings_nkd.xls')
tradings_nkd_1 = pd.read_excel(f'./data/Input/tradings_nkd_1.xls')
tradings_nkd_2 = pd.read_excel(f'./data/Input/tradings_nkd_2.xls')
tradings_nkd_3 = pd.read_excel(f'./data/Input/tradings_nkd_3.xls')

tradings_nkd = pd.concat([tradings_nkd, tradings_nkd_1, tradings_nkd_2, tradings_nkd_3], axis=0)
tradings_nkd = tradings_nkd[['ISIN код эмиссии', 'Накопленный купонный доход', 'Непогашенный номинал']]\
.rename(columns={
    'ISIN код эмиссии': "ISIN", 
    'Накопленный купонный доход': "Interest", 
    'Непогашенный номинал': "Principal"
})
tradings_nkd = tradings_nkd.groupby("ISIN").first()

In [ ]:
Prices.update(tradings_nkd)

In [ ]:
tradings_new = pd.read_excel(f'./data/Input/tradings_new_{REPORT_DATE_STR}.xls')

tradings_new = tradings_new[['Код ISIN', 'Эмиссия', 'Дата', 'Bid, закр.', 'НКД']]\
.rename(columns={
    'Код ISIN': 'ISIN',
    'Эмиссия': 'Name',
    'Дата': 'Date',
    'Bid, закр.': 'Clean price pct',
    'НКД': 'Interest'
})
tradings_new["Max date"] = tradings_new.groupby('ISIN')["Date"].transform('max')
tradings_new = tradings_new.groupby('ISIN').first()

In [ ]:
emissions = pd.read_excel(f'./data/Input/emissions_{REPORT_DATE_STR}.xls')

emissions = emissions[['Валюта', 'код ISIN', 'Непогашенный номинал']]\
.rename(columns={
    'код ISIN': 'ISIN',
    'Валюта': 'Currency',
    'Непогашенный номинал': 'Principal'
})
emissions = emissions[~((emissions["ISIN"].isna()) | (emissions["Principal"].isna()) | (emissions["Principal"]==0))].set_index("ISIN")
tradings_new = tradings_new.merge(emissions, how="left", left_index=True, right_index=True)

In [ ]:
Prices.update(tradings_new)

In [ ]:
Prices.loc[Prices['Interest'] > 0.2*Prices['Principal'], 'Interest'] = 0
Prices['Clean price'] = Prices['Clean price pct']/100 * Prices['Principal']
Prices['Price'] = Prices['Clean price'] + Prices['Interest']

In [ ]:
cbonds_bonds_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[cbonds_bonds_idx, "Type"] = 'Облигация'
Prices.loc[cbonds_bonds_idx, 'Source'] = 'Cbonds'
Prices.loc[cbonds_bonds_idx, 'ISIN in DB'] = 1

In [ ]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[cbonds_bonds_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

##### Шаг 2.2. Stocks

In [ ]:
tradings_stocks_all = pd.read_excel(f'./data/Input/tradings_stocks_all.xls')

tradings_stocks_all = tradings_stocks_all[['ISIN', 'Краткое название эмитента (rus)', 'Валюта номинала', 'Валюта торгов', 'Цена закрытия']]\
.rename(columns={
    'Краткое название эмитента (rus)': 'Name', 
    'Валюта номинала': 'Currency', 
    'Валюта торгов': 'Trade currency', 
    'Цена закрытия': 'Price'
})

In [ ]:
tradings_stocks_all = tradings_stocks_all.merge(CurrencyRate[["currency", "rate"]], how='left', left_on="Trade currency", right_on="currency")\
.rename(columns={'rate': 'trade rate'})
tradings_stocks_all = tradings_stocks_all.merge(CurrencyRate[["currency", "rate"]], how='left', left_on="Currency", right_on="currency")
tradings_stocks_all['RUB Price'] = tradings_stocks_all['Price'] * tradings_stocks_all['trade rate']
tradings_stocks_all['RUB Price'] = tradings_stocks_all.groupby("ISIN")['RUB Price'].transform('mean')
tradings_stocks_all['Currency'] = tradings_stocks_all['Currency'].combine_first(tradings_stocks_all['Trade currency'])
tradings_stocks_all['Clean price'] = tradings_stocks_all['RUB Price'] / tradings_stocks_all['rate']
tradings_stocks_all['Price'] = tradings_stocks_all['Clean price']
tradings_stocks_all['FX Rate'] = tradings_stocks_all['trade rate'].combine_first(tradings_stocks_all['rate'])

In [ ]:
cbonds_stocks_idx = tradings_stocks_all[~tradings_stocks_all["Price"].isna()].index
tradings_stocks_all.loc[cbonds_stocks_idx, "Type"] = 'Акция'
tradings_stocks_all.loc[cbonds_stocks_idx, 'Source'] = 'Cbonds'
tradings_stocks_all.loc[cbonds_stocks_idx, 'ISIN in DB'] = 1

In [ ]:
Prices.update(tradings_stocks_all.groupby("ISIN").first())

In [ ]:
ResultPrices = pd.concat([ResultPrices, Prices[~Prices["Price"].isna()]], axis=0)
Prices = Prices[Prices['Price'].isna()]

#### Шаг 3. MOEX

In [ ]:
HistoryStockBonds = RuDataDF("HistoryStockBonds").df
HistoryStockBonds.head()

In [ ]:
HistoryStockShares = RuDataDF("HistoryStockShares").df
HistoryStockShares.head()

In [ ]:
HistoryStockNdm = RuDataDF("HistoryStockNdm").df
HistoryStockNdm.head()

In [ ]:
HistoryStockCcp = RuDataDF("HistoryStockCcp").df
HistoryStockCcp.head()

In [ ]:
TransformHistoryStockBonds = HistoryStockBonds.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockShares = HistoryStockShares.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockNdm = HistoryStockNdm.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockCcp = HistoryStockCcp.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")

In [ ]:
Prices.update(TransformHistoryStockBonds)
Prices.update(TransformHistoryStockShares)
Prices.update(TransformHistoryStockNdm)
Prices.update(TransformHistoryStockCcp)

In [ ]:
moex_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[moex_idx, 'Source'] = 'RU Data (MOEX)'
Prices.loc[moex_idx, 'ISIN in DB'] = 1

In [ ]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[moex_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

#### Шаг 4. Finish

In [ ]:
FullPrices = pd.concat([ResultPrices, Prices], axis=0)

##### Шаг 4.1. Update currency

In [ ]:
FullPrices = FullPrices\
.reset_index()\
.merge(CurrencyRate[['currency', 'rate']], how='left', left_on='Currency', right_on='currency')\
.set_index("ISIN")

In [ ]:
rub_price_idx = FullPrices[FullPrices["RUB Price"].isna()].index
FullPrices.loc[rub_price_idx, "FX Rate"] = FullPrices.loc[rub_price_idx, "rate"].fillna(1)
FullPrices.loc[rub_price_idx, "RUB Price"] = FullPrices.loc[rub_price_idx, "Price"] * FullPrices.loc[rub_price_idx, "FX Rate"]

In [ ]:
FullPrices.drop(columns=['currency', 'rate'], inplace=True)

##### Шаг 4.2. Write excel

In [ ]:
with pd.ExcelWriter(f'./data/Output/prices_{REPORT_DATE_STR.replace("-", "")}.xlsx', datetime_format='dd.mm.yyyy') as writer: 
    FullPrices[~FullPrices["Price"].isna()].to_excel(writer)

In [ ]:
with pd.ExcelWriter(f'./data/Output/prices_{REPORT_DATE_STR.replace("-", "")}_full.xlsx', datetime_format='dd.mm.yyyy') as writer: 
    FullPrices.to_excel(writer)